<a href="https://colab.research.google.com/github/anukvma/group18_email_subject_generation/blob/main/WIP_PK_QnA_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

# Load Dataset from CSV

In [3]:
!git clone https://github.com/anukvma/group18_email_subject_generation


Cloning into 'group18_email_subject_generation'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 157 (delta 81), reused 33 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (157/157), 3.24 MiB | 6.76 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [67]:
import pandas as pd
df = pd.read_csv('group18_email_subject_generation/sampled_qa_data.csv')
df1 = pd.read_csv('group18_email_subject_generation/unit1_QA.csv')
df1.dropna()
df1.rename(columns={'Question': 'question', 'Answer': 'answer'}, inplace=True)

df3 = pd.read_csv('group18_email_subject_generation/unit3_QA.csv')
df3.dropna()
df3.rename(columns={'Questions': 'question', 'Answers': 'answer'}, inplace=True)

df.drop(columns=['Unnamed: 0', 'unit'], inplace=True)
df.rename(columns={'Q': 'question', 'A': 'answer'}, inplace=True)
df.dropna(inplace=True)

df = pd.concat([df,df1,df3])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 806 entries, 0 to 80
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  806 non-null    object
 1   answer    806 non-null    object
dtypes: object(2)
memory usage: 18.9+ KB


In [68]:
from sklearn.model_selection import train_test_split
# Split the data into training and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)


In [69]:
# Convert to Hugging Face Dataset format
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

In [73]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

def tokenize_function(examples):
    # Tokenize the input (question) and output (answer)
    inputs = tokenizer(examples['question'], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples['answer'], padding="max_length", truncation=True, max_length=512)
    # Rename the labels and return as a dictionary
    outputs['labels'] = outputs['input_ids']
    outputs['input_ids'] = inputs['input_ids']
    outputs['attention_mask'] = inputs['attention_mask']

    return outputs

# Tokenize the dataset
train_tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
val_tokenized_datasets = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/644 [00:00<?, ? examples/s]

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

# Model Training

In [71]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

# Load the BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=val_tokenized_datasets,  # Use the same dataset for simplicity, ideally, you should split train and eval
    tokenizer=tokenizer
)

# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.768600,0.496550
2,0.278900,0.296121
3,0.228700,0.285863
4,0.209600,0.282019
5,0.250500,0.279743
6,0.164100,0.280801
7,0.188200,0.280594
8,0.256600,0.281477
9,0.203400,0.282181
10,0.155400,0.282924


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=1610, training_loss=0.6353561544270249, metrics={'train_runtime': 1189.8095, 'train_samples_per_second': 5.413, 'train_steps_per_second': 1.353, 'total_flos': 1963351395532800.0, 'train_loss': 0.6353561544270249, 'epoch': 10.0})

# Evaluation and Prediction

In [75]:
# Evaluate the model
results = trainer.evaluate()

# Generate an answer for a new question
def generate_answer(question):
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_length=512)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def print_answer(question):
  answer = generate_answer(question)
  print("Generated Answer:", answer)

print_answer("What is Q-Learning, and how does it work?")
print_answer("How does learning rate affect gradient descent?")
print_answer("What is the difference between supervised and unsupervised learning?")

Generated Answer: Q-Learning is a type of machine learning model that uses Q-means to learn a linear representation of the input data. Q-Learning uses a neural network to learn the relationships between input data points, allowing the model to learn more complex representations of the data.
Generated Answer: Learning rate affects gradient descent by adjusting the size of the input gradient, which can affect the accuracy of the gradient.
Generated Answer: Supervised learning involves training a model on unlabeled data, while unsupervised learning uses a supervised learning model to learn from unseen data.
